In [0]:
!pip install transformers

     |████████████████████████████████| 450kB 6.4MB/s 
     |████████████████████████████████| 1.0MB 34.8MB/s 
     |████████████████████████████████| 870kB 54.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=36d7ba8ad746ff77e3e2375465c83c119226b94ae9011b3d2bf7c200a1fa79f3
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
import argparse
import glob
import json
import logging
import os
import random

In [0]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader,RandomSampler,SequentialSampler,TensorDataset
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm,trange

In [0]:
from transformers import (WEIGHTS_NAME,
                         BertConfig,
                         BertModel,
                         BertTokenizer,)

from transformers import glue_compute_metrics as compute_metrics
from transformers import glue_convert_examples_to_features as convert_examples_to_features
from transformers import glue_output_modes as output_modes
from transformers import glue_processors as processors

from transformers.data.processors.utils import InputExample,DataProcessor

In [0]:
try:
    from torch.utils.tensorboard import SummaryWriter   #version 1.14 or higher
except ImportError:
    from tensorboardX import SummaryWriter

In [0]:
import code
import pickle
from  sklearn.metrics.pairwise import cosine_similarity

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
logger=logging.getLogger(__name__)
MODEL_CLASSES={
    "bert":(BertConfig,BertModel,BertTokenizer),
}

In [0]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [0]:
class FAQProcessor(DataProcessor):
  
    def get_candidates(self,file_dir):
        train_df=pd.read_csv(file_dir)
        candidates=train_df[train_df["is_best"]==1][["title","reply"]]
        self.candidate_title=candidates["title"].tolist()
        self.candidate_reply=candidates["reply"].tolist()
        return self._create_examples(self.candidate_title,"train")
    
    def _create_examples(self,lines,set_type):
        examples=[]
        for (i,line) in enumerate(lines):
            guid="%s-%s" % (set_type,i)
            examples.append(InputExample(guid=guid,text_a=line,text_b=None,label=1))
        return examples

In [0]:
def evaluate(args,model,eval_dataset):
    pooled_outputs=[]
    
    
    eval_sampler=SequentialSampler(eval_dataset)
    eval_dataloader=DataLoader(eval_dataset,sampler=eval_sampler,batch_size=16)
    
    
    logger.info(" Num examples = %d",len(eval_dataset))
    logger.info(" Batch size = %d",16)
    eval_loss=0.0
    nb_eval_steps=0
    preds=None
    out_label_ids=None
    
    for batch in tqdm(eval_dataloader,desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(args["device"]) for t in batch)
        
        with torch.no_grad():
            inputs={"input_ids": batch[0],"attention_mask":batch[1]}
            if args["model_type"]!="distilbert":
                inputs["token_type_ids"]=(
                    batch[2] if args["model_type"] in ["bert","xlnet"] else None
                )
            
            outputs=model(**inputs)
            # 关于这两个outputs
            #
            # 1.sequence_output 代表每个单词的向量,和ELMo出来的一样
            #   size是: batch_size * seq_len * hidden_size, 32*句子里单词数量*768
            #   在计算cosine similarity时, 也是可以用和ELMo方法一样,在seq_len上求平均
            #   理论上这么做比用pooled_output好点,因为[CLS] token不太很能表示句子信息
            #
            # 2.pooled_output代表句子的向量
            #   它其实是sequence output的第一个token(即[CLS])经过一个Linear层和一个tanh层的结果 
            #   size是: batch_size * hidden_size,  32*768
            sequence_output,pooled_output=outputs[:2]
            pooled_outputs.append(pooled_output)
            
    return pooled_outputs  

In [0]:
def load_examples(args,task, tokenizer):
    processor = FAQProcessor()
    # Load data features from cache or dataset file
    # cached_features_file = os.path.join(
    #     args["data_dir"],
    #     "cached_{}_{}_{}".format(
    #         list(filter(None, args["model_name_or_path"].split("/"))).pop(),
    #         str(args["max_seq_length"]),
    #         str(task),
    #     ),
    # )
    logger.info("Creating features from dataset file at %s", args["data_dir"])
    examples = (
        processor.get_candidates(args["data_dir"]) 
    )
    features = convert_examples_to_features(
        examples,
        tokenizer,
        label_list=[1],
        output_mode="classification",
        max_length=args["max_seq_length"],
        pad_on_left=bool(args["model_type"] in ["xlnet"]),  # pad on the left for xlnet
        pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
        pad_token_segment_id=4 if args["model_type"] in ["xlnet"] else 0,
    )


    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)

    dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids)
    return dataset, processor.candidate_title, processor.candidate_reply

In [0]:
def main(args):
    
    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
    )


    set_seed()
    task_name = ""
    model_type = args["model_type"]
    
    
    config_class, model_class, tokenizer_class = MODEL_CLASSES[model_type]
    config = config_class.from_pretrained(
        args["config_name"],
        finetuning_task=task_name, 
        cache_dir=None,
    )
    tokenizer = tokenizer_class.from_pretrained(
        args["tokenizer_name"],
        do_lower_case=True,
        cache_dir=None,
    )
    model = model_class.from_pretrained(
        args["model_name_or_path"],
        from_tf=bool(".ckpt" in args["model_name_or_path"]),
        config=config,
        cache_dir=None,
    )

 
    model.to(args["device"])

    # load dataset
    if not os.path.exists("/content/drive/My Drive/embeddings.pkl"):
        logger.info("Training/evaluation parameters %s", args)

        eval_dataset, candidate_title, candidate_reply = load_examples(args, task_name, tokenizer)
    
        pooled_outputs = evaluate(args, model, eval_dataset)
    
        #把所有candidates embedding拼起来, 每个pooled_output是32, 拼起来一共 18677*768
        candidate_embeddings = torch.cat([o.cpu().data for o in pooled_outputs]).numpy()

        with open("/content/drive/My Drive/embeddings.pkl", "wb") as fout:
            pickle.dump([candidate_title, candidate_reply, candidate_embeddings], fout)

            
    else:
        with open("/content/drive/My Drive/embeddings.pkl", "rb") as fin:
            candidate_title, candidate_reply, candidate_embeddings = pickle.load(fin)

    while True:
        title = input("你的问题是？\n")
        if len(title.strip()) == 0:
            continue
        
        #[CLS]句子[SEP]None[SEP] 只用了一个句子
        examples = [InputExample(guid=0, text_a=title, text_b=None, label=1)]
        
        
        features = convert_examples_to_features(
            examples,
            tokenizer,
            label_list=[1],
            output_mode="classification",
            max_length=args["max_seq_length"],
            pad_on_left=bool(args["model_type"] in ["xlnet"]),  # pad on the left for xlnet
            pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
            pad_token_segment_id=4 if args["model_type"] in ["xlnet"] else 0,
        )

        # Convert to Tensors and build dataset
        all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
        all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
        all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)

        dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids)
        pooled_outputs = evaluate(args, model, dataset)
        title_embedding = torch.cat([o.cpu().data for o in pooled_outputs]).numpy()

        scores = cosine_similarity(title_embedding, candidate_embeddings)[0]
        top5_indices = scores.argsort()[-5:][::-1]
        for index in top5_indices:
            print("可能的答案，参考问题：" + candidate_title[index] + "\t答案：" + candidate_reply[index] + "\t得分：" + str(scores[index]))
            print()


In [0]:
args={
    "model_type":"bert",
    "data_dir": "/content/drive/My Drive/Data/lawzhidao_filter.csv",
    "model_name_or_path": "/content/drive/My Drive/chinese_wwm_ext_pytorch/",
    "config_name": "/content/drive/My Drive/chinese_wwm_ext_pytorch/",
    "tokenizer_name": "/content/drive/My Drive/chinese_wwm_ext_pytorch/",
    "do_train":False,
    "do_eval":False,
    "evaluate_during_training":False,
    "do_lower_case":False,
    "per_gpu_train_batch_size":8,
    "per_gpu_eval_batch_size":8,
    "gradient_accumulation_steps":1,
    "learning_rate":5e-5,
    "adam_epsilon":1e-8,
    "max_grad_norm":1.0,
    "weight_decay":0.0,
    "max_grad_norm":1.0,
    "max_seq_length":128,
    "device":"cpu",
    
    
}


if torch.cuda.is_available():
    args["device"]="cuda"

main(args)

01/27/2020 10:11:37 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/chinese_wwm_ext_pytorch/config.json
01/27/2020 10:11:37 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": "",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "pruned_heads"

你的问题是？
怎样找律师?


01/27/2020 10:12:07 - INFO - transformers.data.processors.glue -   Writing example 0
01/27/2020 10:12:07 - INFO - transformers.data.processors.glue -   *** Example ***
01/27/2020 10:12:07 - INFO - transformers.data.processors.glue -   guid: 0
01/27/2020 10:12:07 - INFO - transformers.data.processors.glue -   input_ids: 101 2582 3416 2823 2526 2360 136 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
01/27/2020 10:12:07 - INFO - transformers.data.processors.glue -   attention_mask: 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
01/27/2020 10:12:07 - INFO - transformers.data.processors.glue 

可能的答案，参考问题：笔迹鉴定怎么收费的?	答案：每个省和每个地方的收费标准都不一样。都是有自己所在省司法厅和发改委指导定价。都是有收费明细的。而且笔迹鉴定也分很多种：变造伪造篡改、后期添加等等。	得分：0.987727

可能的答案，参考问题：工资保密协议如何写?	答案：举例如下：工资保密协议甲方：乙方：为了保护甲乙双方的权益，保守双方的薪酬秘密，维护和谐的工作氛围，促进双方的稳定发展，甲、乙双方在遵循诚实信用、平等自愿的原则下，经协商就薪酬保密事项达成协议，条款如下：一、甲方的工资薪酬体系设置属于甲方的管理信息内容之一，并经甲方采取保密措施，属于甲方的商业秘密。乙方所接触的工资薪酬信息未经甲方允许不得向第三方透露。乙方必须严格遵守甲方的保密制度，不得询问与自己薪酬无关的情况，不得泄漏本人的薪酬待遇情况，不得以任何形式向他人询问薪酬状况。非本人原因知悉他人薪酬情况者，应及时向甲方主管报告，并采取措施防止泄密进一步扩大，不得传播、散步、比对。二、若乙方作为管理人员及相关工作人员参与或接触薪酬管理或工资作业，要严格遵守公司薪酬保密制度，不得向第三方泄漏所知悉的工资薪酬情况，不得在任何非作业场所公开谈论与薪酬有关的事宜，不得在与薪酬作业无关的人员面前进行操作。带有工资信息的资料必须妥善保管，需存放入文件柜或抽屉，不得随意乱放。废弃不用的资料信息，需用粉碎机进行粉碎处理。三、甲方要对乙方的工资薪酬情况进行保密，非因履行工作职责之需要不得向第三方透露。四、违约责任约定：本协议约定的工资保密事项是公司的重要纪律。违反本协议约定的，一经核实给予违纪解除劳动合同的处理。五、本协议正本一式两份，甲乙双方各执一份，作为劳动合同附件，与劳动合同具有同等法律效力。本协议经甲、乙双方签字或盖章之日起生效。甲方（盖章或签字）：乙方（签字）：日期：年月日日期：年月日	得分：0.9876441

可能的答案，参考问题：怎样快速办理离婚手续？	答案：办理协议离婚手续应持那些证件或证明材料？申请离婚登记当事*双方应持：本*的户口簿（95新户口簿）、居*身份证（应在有效期内）、结婚证、离婚协议书（A4纸打印一式3份）、双方各2张2寸近期半身免冠证件照片（同一底板）、《申请离婚登记声明书》。协议离婚手续怎么办？协议离婚必须办理离婚登记。离婚登记是夫妻双方自愿离婚的必经程序。他是由婚姻登记机关依照行*程序

01/27/2020 10:12:34 - INFO - transformers.data.processors.glue -   Writing example 0
01/27/2020 10:12:34 - INFO - transformers.data.processors.glue -   *** Example ***
01/27/2020 10:12:34 - INFO - transformers.data.processors.glue -   guid: 0
01/27/2020 10:12:34 - INFO - transformers.data.processors.glue -   input_ids: 101 2582 720 2823 2526 2360 136 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
01/27/2020 10:12:34 - INFO - transformers.data.processors.glue -   attention_mask: 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
01/27/2020 10:12:34 - INFO - transformers.data.processors.glue -

...

可能的答案，参考问题：无证驾驶被捉了怎么办	答案：1、新交法里无证驾驶处罚：未取得机动车驾驶证、机动车驾驶证被吊销或者机动车驾驶证被暂扣期间驾驶机动车的处二百元以上二千元以下罚款，可以并处十五日以下拘留。对于未随身携带机动车驾驶证的，无论驾驶人在行驶过程中违法与否，均不得继续驾驶该车辆。公安机关交通管理部门应当扣留机动车，通知驾驶人提供驾驶证。根据所提供驾驶证的不同情形分别进行处理。a)如若提供的驾驶证真实、合法并且在驾驶时无任何违法行为的，给予警告或20-200元的罚款，及时退还机动车;b)如若提供的驾驶证真实、合法但在驾驶时有违法行为的，根据其违法的情况和未随身携带驾驶证一并予以处罚，并及时退还机动车;c)如驾驶人不能提供真实、合法的机动车驾驶证，视为无证驾驶，可按照以上情形处罚。2、无证驾驶撞人的处罚规定无证驾驶撞人，构成交通肇事罪，一般来说没有加重情节的话应该判刑3年以下。根据《中华人民共和国刑法》第一百三十三条规定，违反交通运输管理法规，因而发生重大事故，致人重伤、死亡或者使公私财产遭受重大损失的，处三年以下有期徒刑或者拘役;交通运输肇事后逃逸或者有其他特别恶劣情节的，处三年以上七年以下有期徒刑;因逃逸致人死亡的，处七年以上有期徒刑。	得分：0.9884206

可能的答案，参考问题：卖淫二次被抓怎么办	答案：光是卖淫的话，是不构成犯罪的，但会受到治安处罚，由公安机关处10日以上15日以下拘留，可以并处5000元以下罚款；情节较轻的，处5日以下拘留或者500元以下罚款。除依法治安处罚外，还可以由公安机关决定收容教育。收容教育期限为六个月至二年。但是，明知自己患有梅毒、淋病等严重性病卖淫的，构成传播性病罪，处五年以下有期徒刑、拘役或者管制，并处罚金。	得分：0.9879911

可能的答案，参考问题：被抓酒驾怎么办	答案：饮酒驾驶：饮酒驾驶机动车辆，罚款1000元—2000元、记12分并暂扣驾照6个月;饮酒驾驶营运机动车，罚款5000元，记12分，处以15日以下拘留，并且5年内不得重新获得驾照。醉酒驾驶：醉酒驾驶机动车辆，吊销驾照，5年内不得重新获取驾照，经过判决后处以拘役，并处罚金;醉酒驾驶营运机动车辆，吊销驾照，10年内不得重新获取驾照，终生不得驾驶营运车辆，经过判决后处以拘役，并处罚金。第九十一条，饮酒后驾驶机动车的，处暂扣六个月机动车驾驶证，并处一千

01/27/2020 10:13:25 - INFO - transformers.data.processors.glue -   Writing example 0
01/27/2020 10:13:25 - INFO - transformers.data.processors.glue -   *** Example ***
01/27/2020 10:13:25 - INFO - transformers.data.processors.glue -   guid: 0
01/27/2020 10:13:25 - INFO - transformers.data.processors.glue -   input_ids: 101 2582 720 976 5273 4173 5489 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
01/27/2020 10:13:25 - INFO - transformers.data.processors.glue -   attention_mask: 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
01/27/2020 10:13:25 - INFO - transformers.data.processors.glue -

可能的答案，参考问题：五险一金是什么	答案：五险一金是指用人单位给予劳动者的几种保障性待遇的合称，包括养老保险、医疗保险、失业保险、工伤保险和生育保险，还有住房公积金。养老保险、医疗保险和失业保险，这三种险和住房公积金是由企业和个人共同缴纳的保费，工伤保险和生育保险完全是由企业承担的，个人不需要缴纳。缴纳社会保险是国家强制性的要求，公积金，是根据企业效益而定，效益不好，可以自己确定比例，不属于社会保险范畴。五险一金的作用：1.养老保险‍是为了老有所养，劳动者到了法定退休年龄可以领取退休金；2.医疗保险是职工生病作为医疗报销用的；3.失业保险是职工失业后，领取生活费的；4.生育保险是职工生育作为生育报销和有关待遇的；5.工伤保险是职工发生工伤，支付医疗费用或者赔偿有关工伤待遇的；6.住房公积金是住房改革，企业和劳动者各缴纳一定的住房基金，符合提取条件可以提取，也可以再购买商品房时享受更低的住房贷款利息，减轻劳动者经济负担。	得分：0.9756167

可能的答案，参考问题：复婚和再婚的区别是什么	答案：复婚是指已离婚的男女双方自愿恢复夫妻关系，到婚姻登记机关办理登记手续，重新确立婚姻关系的行为。复婚是一种法律行为，如果男女双方离婚后又自愿要求恢复夫妻关系的，说明双方的感情并未完全破裂，或造成离婚的原因已被消除。在这种情况下，法律允许双方重新确立婚姻关系。但是，婚姻关系是一种法律关系，它的产生与消除都必须经过法定的程序，复婚同结婚所要履行的手续一样，都要到婚姻登记机关办理登记手续。所不同的是，办理复婚登记手续后，要将原离婚证或法院判决书(或调解书)撤销，发给复婚登记证，才能发生法律效力。复婚必须是男女双方出于自愿，不是强迫的或他人的干涉，复婚时，男女双方必须持有关证明一同到婚姻登记机关办理复婚登记手续。婚姻登记机关要进行审查，凡是符合法定条件的，就发给复婚登记证，这时，合法的夫妻关系便恢复了。但是，如果双方不经复婚登记，私下同居，就不能取得合法的夫妻身份，也不能受到法律的保护。再婚是指一方在配偶死后或双方离婚以后与他人再次结婚的行为。再婚是以终止原来的婚姻关系为前提的，如果不终止原婚姻关系又结婚，则构成重婚。	得分：0.9750802

可能的答案，参考问题：出租房子合同怎么写	答案：签订租房合同应注意下列事项：1、双方当事人的情况合同中应写明出租人和承租人的姓名及住址

01/27/2020 10:13:49 - INFO - numexpr.utils -   NumExpr defaulting to 2 threads.


KeyboardInterrupt: ignored

In [0]:
def mean_reciprocal_rank(rs):
    rs = (np.asarray(r).nonzero()[0] for r in rs)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])


def evaluate_mmr(args):
    # load model
    set_seed()
    task_name = ""
    model_type = args["model_type"]
    
    
    config_class, model_class, tokenizer_class = MODEL_CLASSES[model_type]
    config = config_class.from_pretrained(
        args["config_name"],
        finetuning_task=task_name, 
        cache_dir=None,
    )
    tokenizer = tokenizer_class.from_pretrained(
        args["tokenizer_name"],
        do_lower_case=True,
        cache_dir=None,
    )
    model = model_class.from_pretrained(
        args["model_name_or_path"],
        from_tf=bool(".ckpt" in args["model_name_or_path"]),
        config=config,
        cache_dir=None,
    )
    model.to(args["device"])
    
    
    
    # load candidate embeddings
    with open("/content/drive/My Drive/embeddings.pkl", "rb") as fin:
        candidate_title, candidate_reply, candidate_embeddings = pickle.load(fin)
    
    
    # load test data
    df=pd.read_csv(args["data_dir"])
    questions=df["question"].tolist()
    matched_questions=df["title"].tolist()
    matched_questions_index = []
    for q in matched_questions:
        flg = False
        for i, _q in enumerate(candidate_title):
            if q == _q:
                matched_questions_index.append([i])
                flg = True
                break
        if flg == False:
            matched_questions_index.append([-1])
    
    matched_questions_index = np.asarray(matched_questions_index)
    
    
    #convert questions in test data to BERT input
    examples = [InputExample(guid=0, text_a=title, text_b=None, label=1) for title in questions]

    
    features = convert_examples_to_features(
        examples,
        tokenizer,
        label_list=[1],
        output_mode="classification",
        max_length=args["max_seq_length"],
        pad_on_left=bool(args["model_type"] in ["xlnet"]),  # pad on the left for xlnet
        pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
        pad_token_segment_id=4 if args["model_type"] in ["xlnet"] else 0,
    )
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)


    dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids)
    sequence_outputs = evaluate(args, model, dataset)
    question_embedding = torch.cat([o.cpu() for o in sequence_outputs]).numpy()


    
    
    scores = cosine_similarity(question_embedding, candidate_embeddings)
    sorted_indices = scores.argsort()[:, ::-1]#[-5:][::-1]
    # code.interact(local=locals())
    mmr = mean_reciprocal_rank(sorted_indices==matched_questions_index)
    print("mean reciprocal rank: {}".format(mmr))


In [0]:
args={
    "model_type":"bert",
    "data_dir": "/content/drive/My Drive/Data/lawzhidao_evaluate.csv",
    "model_name_or_path": "/content/drive/My Drive/chinese_wwm_ext_pytorch/",
    "config_name": "/content/drive/My Drive/chinese_wwm_ext_pytorch/",
    "tokenizer_name": "/content/drive/My Drive/chinese_wwm_ext_pytorch/",
    "do_train":False,
    "do_eval":False,
    "evaluate_during_training":False,
    "do_lower_case":False,
    "per_gpu_train_batch_size":8,
    "per_gpu_eval_batch_size":8,
    "gradient_accumulation_steps":1,
    "learning_rate":5e-5,
    "adam_epsilon":1e-8,
    "max_grad_norm":1.0,
    "weight_decay":0.0,
    "max_grad_norm":1.0,
    "max_seq_length":128,
    "device":"cpu",
    
    
}


if torch.cuda.is_available():
    args["device"]="cuda"
    
    
evaluate_mmr(args)

01/20/2020 08:09:31 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/chinese_wwm_ext_pytorch/config.json
01/20/2020 08:09:31 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": "",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "pruned_heads"

mean reciprocal rank: 0.1838426239890649
